In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('Datasets/Train/Train.csv')

In [3]:
data.head(n=10)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,3.0,0.0,"O'Donoghue, Ms. Bridget",female,NaN,0.0,0.0,364856,7.7500,NaN,Q,NaN,NaN,NaN
1,2.0,0.0,"Morley, Mr. Henry Samuel (""Mr Henry Marshall"")",male,39.0,0.0,0.0,250655,26.0000,NaN,S,NaN,NaN,NaN
2,2.0,1.0,"Smith, Miss. Marion Elsie",female,40.0,0.0,0.0,31418,13.0000,NaN,S,9,NaN,NaN
3,3.0,1.0,"Goldsmith, Mrs. Frank John (Emily Alice Brown)",female,31.0,1.0,1.0,363291,20.5250,NaN,S,C D,NaN,"Strood, Kent, England Detroit, MI"
4,3.0,1.0,"McCoy, Miss. Agnes",female,NaN,2.0,0.0,367226,23.2500,NaN,Q,16,NaN,NaN
5,2.0,0.0,"Gaskell, Mr. Alfred",male,16.0,0.0,0.0,239865,26.0000,NaN,S,NaN,NaN,"Liverpool / Montreal, PQ"
6,2.0,0.0,"Phillips, Mr. Escott Robert",male,43.0,0.0,1.0,S.O./P.P. 2,21.0000,NaN,S,NaN,NaN,"Ilfracombe, Devon"
7,1.0,1.0,"Leader, Dr. Alice (Farnham)",female,49.0,0.0,0.0,17465,25.9292,D17,S,8,NaN,"New York, NY"
8,1.0,0.0,"Brandeis, Mr. Emil",male,48.0,0.0,0.0,PC 17591,50.4958,B10,C,NaN,208.0,"Omaha, NE"
9,2.0,0.0,"Wheeler, Mr. Edwin ""Frederick""",male,NaN,0.0,0.0,SC/PARIS 2159,12.8750,NaN,S,NaN,NaN,NaN


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009 entries, 0 to 1008
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1009 non-null   float64
 1   survived   1009 non-null   float64
 2   name       1009 non-null   object 
 3   sex        1009 non-null   object 
 4   age        812 non-null    float64
 5   sibsp      1009 non-null   float64
 6   parch      1009 non-null   float64
 7   ticket     1009 non-null   object 
 8   fare       1008 non-null   float64
 9   cabin      229 non-null    object 
 10  embarked   1008 non-null   object 
 11  boat       374 non-null    object 
 12  body       98 non-null     float64
 13  home.dest  582 non-null    object 
dtypes: float64(7), object(7)
memory usage: 110.5+ KB


In [5]:
columns_to_drop = ["ticket","cabin","embarked","name",'boat','body','home.dest']
data_clean = data.drop(columns_to_drop,axis=1)

In [6]:
from sklearn.preprocessing import LabelEncoder

le  = LabelEncoder()

data_clean["sex"] = le.fit_transform(data_clean["sex"])

In [7]:
data_clean.head()

,pclass,survived,sex,age,sibsp,parch,fare
0,3.0,0.0,0,NaN,0.0,0.0,7.750
1,2.0,0.0,1,39.0,0.0,0.0,26.000
2,2.0,1.0,0,40.0,0.0,0.0,13.000
3,3.0,1.0,0,31.0,1.0,1.0,20.525
4,3.0,1.0,0,NaN,2.0,0.0,23.250


In [8]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009 entries, 0 to 1008
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1009 non-null   float64
 1   survived  1009 non-null   float64
 2   sex       1009 non-null   int64  
 3   age       812 non-null    float64
 4   sibsp     1009 non-null   float64
 5   parch     1009 non-null   float64
 6   fare      1008 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 55.3 KB


In [9]:
data_clean = data_clean.fillna(data_clean["age"].mean())

In [10]:
input_cols = ["pclass","sex","age","sibsp","parch","fare"]
output_cols = ["survived"]

X = data_clean[input_cols]
Y = data_clean[output_cols]

print(X.shape,Y.shape)
print(type(X))


(1009, 6) (1009, 1)
<class 'pandas.core.frame.DataFrame'>


In [11]:
def entropy(col):
    counts = np.unique(col,return_counts=True)
    N = float(col.shape[0])
    ent= 0.0
    for ix in counts[1]:
        p = ix/N
        ent+=(-1.0*p*np.log2(p))
        
    return ent
        

In [12]:
col = np.array([1,1,1,1,1,1,1,])
entropy(col)

0.0

In [13]:
def divide_data(x_data,fkey,fval):
    x_right = pd.DataFrame([],columns = x_data.columns)
    x_left = pd.DataFrame([],columns = x_data.columns)
    
    for ix in range(x_data.shape[0]):
        val = x_data[fkey].loc[ix]
        
        if val > fval:
            x_right = x_right.append(x_data.loc[ix])
            
        else:
            x_left = x_left.append(x_data.loc[ix])
            
    return x_left,x_right

In [14]:
# x_left, x_right = divide_data(data_clean[:10],"Sex",0.5)
# print(x_left,"\n",x_right)

In [15]:
def information_gain(x_data,fkey,fval):
    left,right = divide_data(x_data,fkey,fval)
    l = float(left.shape[0])/x_data.shape[0]
    r = float(right.shape[0])/x_data.shape[0]
    
    if(left.shape[0]==0 or right.shape[0]==0):
        return -1000000
    i_gain = entropy(x_data.survived) - (l*entropy(left.survived) + r*(entropy(right.survived)))
    return i_gain

In [16]:
for fx in X.columns:
    print(fx)
    print(information_gain(data_clean,fx,data_clean[fx].mean()))

pclass
0.055456910002982474
sex
0.19274737190850932
age
0.001955929827450964
sibsp
0.006492394392888956
parch
0.019756080122948272
fare
0.04242793401428169


In [17]:
class DecisionTree:
    def __init__(self,depth = 0,max_depth = 5):
        self.left = None
        self.right = None
        self.fkey = None
        self.fval = None
        self.max_depth = max_depth
        self.depth = depth
        self.target = None
        
    def train(self,X_train):
        features = ['pclass','sex','age','sibsp','parch','fare']
        info_gains = []
        
        for ix in features:
            i_gain = information_gain(X_train,ix,X_train[ix].mean())
            info_gains.append(i_gain)
            
        self.fkey = features[np.argmax(info_gains)]
        self.fval = X_train[self.fkey].mean()
        print("Making Tree features is", self.fkey)
        
        data_left, data_right = divide_data(X_train,self.fkey,self.fval)
        data_left = data_left.reset_index(drop=True)
        data_right = data_right.reset_index(drop=True)
        
        if data_left.shape[0] == 0 or data_right.shape[0] == 0:
            if X_train.survived.mean() >=0.5:
                self.target = "Survive"
            else:
                self.target = "Dead"
        
        
        if self.depth>=self.max_depth:
            if X_train.survived.mean() >= 0.5:
                self.target = "Survive"
            else:
                self.target = "Dead"
            return
        
        self.left = DecisionTree(depth = self.depth+1,max_depth = self.max_depth)
        self.left.train(data_left)
        
        self.right = DecisionTree(depth = self.depth+1,max_depth = self.max_depth)
        self.right.train(data_right)
        
        if X_train.survived.mean() >=0.5:
            self.target = "Survive"
        else:
            self.target = "Dead"
        return
    
    def predict(self,test):
        if test[self.fkey]>self.fval:
            if self.right is None:
                return self.target
            return self.right.predict(test)
        else:
            if self.left is None:
                return self.target
            return self.left.predict(test)

In [18]:
d = DecisionTree()


In [19]:
split = int(1.0*data_clean.shape[0])
train_data = data_clean[:split]
test_data = data_clean[split:]
test_data = test_data.reset_index(drop=True)

In [20]:
print(train_data.shape,test_data.shape)

(1009, 7) (0, 7)


In [21]:
dt = DecisionTree()

In [22]:
dt.train(train_data)

Making Tree features is sex
Making Tree features is pclass
Making Tree features is pclass
Making Tree features is fare
Making Tree features is sibsp
Making Tree features is age
Making Tree features is age
Making Tree features is sibsp
Making Tree features is age
Making Tree features is sibsp
Making Tree features is parch
Making Tree features is fare
Making Tree features is sibsp
Making Tree features is fare
Making Tree features is age
Making Tree features is age
Making Tree features is parch
Making Tree features is parch
Making Tree features is sibsp
Making Tree features is fare
Making Tree features is fare
Making Tree features is fare
Making Tree features is age
Making Tree features is sibsp
Making Tree features is sibsp
Making Tree features is fare
Making Tree features is fare
Making Tree features is age
Making Tree features is fare
Making Tree features is fare
Making Tree features is fare
Making Tree features is age
Making Tree features is fare
Making Tree features is parch
Making T

In [ ]:
print(dt.fkey)
print(dt.fval)
print(dt.left.fkey)
print(dt.right.fkey)


In [24]:
print(test_data.head())

Empty DataFrame
Columns: [Survived, Pclass, Sex, Age, SibSp, Parch, Fare]
Index: []


In [25]:
y_pred = []
for ix in range(test_data.shape[0]):
    y_pred.append(dt.predict(test_data.loc[ix]))

In [26]:
y_actual = test_data[output_cols]


In [27]:
le = LabelEncoder()
y_pred = le.fit_transform(y_pred)

In [28]:
print(y_pred)

[]


In [29]:
y_pred = np.array(y_pred.reshape((-1,1)))
print(y_pred.shape)

(0, 1)


In [30]:
acc = np.sum(np.array(y_pred) == np.array(y_actual))/y_pred.shape[0]

/home/insanenerd/myvenv/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in long_scalars
  """Entry point for launching an IPython kernel.


In [31]:
print(acc)

nan


In [32]:
from sklearn.tree import DecisionTreeClassifier

In [33]:
sk_tree = DecisionTreeClassifier(criterion='entropy',max_depth = 3)

In [34]:
sk_tree.fit(train_data[input_cols],train_data[output_cols])

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=3, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [35]:
sk_tree.predict(test_data[input_cols])

ValueError: Found array with 0 sample(s) (shape=(0, 6)) while a minimum of 1 is required.

In [ ]:
sk_tree.score(test_data[input_cols],test_data[output_cols])

In [ ]:
import pydotplus
from sklearn.externals.six import StringIO
from IPython.display import Image
from sklearn.tree import export_graphviz

In [ ]:
dot_data = StringIO()
export_graphviz(sk_tree, out_file=dot_data, filled=True,rounded=True)

In [ ]:
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(criterion='entropy',max_depth = 5)

In [ ]:
rf.fit(train_data[input_cols],train_data[output_cols])

In [ ]:
rf.score(train_data[input_cols],train_data[output_cols])

In [ ]:
rf.score(test_data[input_cols],test_data[output_cols])

In [ ]:
acc = cross_val_score(RandomForestClassifier(criterion='entropy',max_depth = 4),train_data[input_cols],train_data[output_cols],cv=5).mean()

In [ ]:
print(acc)

In [ ]:
acc_list = []
for i in range(1,50):
    acc = cross_val_score(RandomForestClassifier(n_estimators=i,criterion='entropy',max_depth = 4),train_data[input_cols],train_data[output_cols],cv=5).mean()
    acc_list.append(acc)

In [ ]:
print(acc_list)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(acc_list)
plt.show()

In [ ]:
rf = RandomForestClassifier(n_estimators=22,criterion='entropy',max_depth = 5)
rf.fit(train_data[input_cols],train_data[output_cols])

In [ ]:
rf.score(train_data[input_cols],train_data[output_cols])

In [ ]:
rf.score(test_data[input_cols],test_data[output_cols])

In [23]:
test = pd.read_csv('Datasets/Test/Test.csv')


In [24]:
print(test.head(10))

   pclass                                         name     sex   age  sibsp  \
0     1.0             Flynn, Mr. John Irwin ("Irving")    male  36.0    0.0   
1     3.0                 Sage, Miss. Constance Gladys  female   NaN    8.0   
2     1.0                        Rood, Mr. Hugh Roscoe    male   NaN    0.0   
3     2.0                 Gillespie, Mr. William Henry    male  34.0    0.0   
4     2.0                   Collander, Mr. Erik Gustaf    male  28.0    0.0   
5     2.0  Collyer, Mrs. Harvey (Charlotte Annie Tate)  female  31.0    1.0   
6     1.0                     Uruchurtu, Don. Manuel E    male  40.0    0.0   
7     3.0                             Razi, Mr. Raihed    male   NaN    0.0   
8     3.0                             Davies, Mr. Evan    male  22.0    0.0   
9     1.0                   Loring, Mr. Joseph Holland    male  30.0    0.0   

   parch       ticket     fare cabin embarked boat  body  \
0    0.0     PC 17474  26.3875   E25        S    5   NaN   
1    2.0  

In [25]:
test_clean = test.drop(columns_to_drop,axis=1)


In [26]:
print(test_clean.head())

   pclass     sex   age  sibsp  parch     fare
0     1.0    male  36.0    0.0    0.0  26.3875
1     3.0  female   NaN    8.0    2.0  69.5500
2     1.0    male   NaN    0.0    0.0  50.0000
3     2.0    male  34.0    0.0    0.0  13.0000
4     2.0    male  28.0    0.0    0.0  13.0000


In [27]:
arr = np.zeros(test_clean.shape[0])
test_clean = test_clean.assign(survived = arr)
test_clean = test_clean[['survived','pclass','sex','age','sibsp','parch','fare']]


In [28]:
test_clean["sex"] = le.fit_transform(test_clean["sex"])
data_clean = data_clean.fillna(data_clean["age"].mean())

In [32]:
output = []

invdict = {'Survive':1.0,'Dead':0.0}
for i in range(test_clean.shape[0]):
    a = invdict[dt.predict((test_clean.loc[i]))]
    
    output.append(a)
df = pd.DataFrame(output)
df.to_csv('mypred.csv')